# TCGA-BRCA Risk Data Inventory
This section imports the Python packages used for data loading, GDC API calls, survival analysis, and plotting.

In [ ]:
from pathlib import Path
import json

import pandas as pd
import requests
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import multivariate_logrank_test, pairwise_logrank_test

This section resolves project-relative Windows paths with `pathlib.Path`, defines input/output locations, and creates output directories.

In [ ]:
input_rel = Path(r"outputs\brca_subtyping\tables\brca_wsi_pam50_case_labels.csv")

candidate_roots = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
project_root = next((root for root in candidate_roots if (root / input_rel).exists()), None)
if project_root is None:
    raise FileNotFoundError(f"Could not locate input file at {input_rel} from current working directory tree.")

labels_path = project_root / input_rel
survival_out_path = project_root / Path(r"outputs\brca_risk\tables\brca_survival_labels_clean.csv")
risk_out_path = project_root / Path(r"outputs\brca_risk\tables\brca_risk_pam50_merged.csv")
km_fig_path = project_root / Path(r"outputs\brca_risk\figures\km_pam50.png")

survival_out_path.parent.mkdir(parents=True, exist_ok=True)
risk_out_path.parent.mkdir(parents=True, exist_ok=True)
km_fig_path.parent.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Input labels: {labels_path}")
print(f"Survival output: {survival_out_path}")
print(f"Risk output: {risk_out_path}")
print(f"KM figure output: {km_fig_path}")

This section loads the PAM50 label table, keeps only `case_id` and `pam50_subtype`, removes duplicate case IDs, and prints subtype availability.

In [ ]:
pam50_labels = (
    pd.read_csv(labels_path, usecols=["case_id", "pam50_subtype"])
    .dropna(subset=["case_id", "pam50_subtype"])
    .drop_duplicates(subset=["case_id"])
)

print("PAM50 label shape:", pam50_labels.shape)
print(pam50_labels["pam50_subtype"].value_counts())

This section fetches expanded TCGA-BRCA clinical case records from the GDC `/cases` endpoint using pagination and the requested fields.

In [ ]:
endpoint = "https://api.gdc.cancer.gov/cases"
fields = [
    "case_id",
    "demographic.vital_status",
    "demographic.days_to_death",
    "diagnoses.days_to_last_follow_up",
    "follow_ups.days_to_follow_up",
]
filters = {
    "op": "and",
    "content": [
        {
            "op": "in",
            "content": {
                "field": "project.project_id",
                "value": ["TCGA-BRCA"],
            },
        }
    ],
}

all_hits = []
from_idx = 0
page_size = 500

while True:
    params = {
        "filters": json.dumps(filters),
        "fields": ",".join(fields),
        "format": "JSON",
        "from": from_idx,
        "size": page_size,
    }
    response = requests.get(endpoint, params=params, timeout=60)
    response.raise_for_status()

    payload = response.json()["data"]
    hits = payload.get("hits", [])
    all_hits.extend(hits)

    total = payload.get("pagination", {}).get("total", len(all_hits))
    from_idx += page_size
    if len(all_hits) >= total or not hits:
        break

print(f"Fetched {len(all_hits)} case records from GDC.")

This section converts nested clinical fields into one row per case, builds `event` and `time_days`, and removes missing or non-positive survival times.

In [ ]:
def to_number(value):
    if value is None:
        return None
    if isinstance(value, (int, float)):
        return float(value)
    if isinstance(value, str):
        text = value.strip()
        if not text:
            return None
        try:
            return float(text)
        except ValueError:
            return None
    return None


records = []
for hit in all_hits:
    case_id = hit.get("case_id")

    demographic = hit.get("demographic") or {}
    vital_status = (demographic.get("vital_status") or "").strip().lower()
    event = 1 if vital_status in {"dead", "deceased"} else 0
    days_to_death = to_number(demographic.get("days_to_death"))

    diagnosis_values = []
    for diagnosis in hit.get("diagnoses") or []:
        if isinstance(diagnosis, dict):
            value = to_number(diagnosis.get("days_to_last_follow_up"))
            if value is not None:
                diagnosis_values.append(value)
    days_to_last_follow_up = max(diagnosis_values) if diagnosis_values else None

    follow_up_values = []
    for follow_up in hit.get("follow_ups") or []:
        if isinstance(follow_up, dict):
            value = to_number(follow_up.get("days_to_follow_up"))
            if value is not None:
                follow_up_values.append(value)
    days_to_follow_up = max(follow_up_values) if follow_up_values else None

    if event == 1:
        time_days = days_to_death
    else:
        candidates = [v for v in [days_to_last_follow_up, days_to_follow_up] if v is not None]
        time_days = max(candidates) if candidates else None

    records.append(
        {
            "case_id": case_id,
            "vital_status": vital_status,
            "event": event,
            "days_to_death": days_to_death,
            "days_to_last_follow_up": days_to_last_follow_up,
            "days_to_follow_up": days_to_follow_up,
            "time_days": time_days,
        }
    )

brca_survival = pd.DataFrame.from_records(records)
brca_survival["time_days"] = pd.to_numeric(brca_survival["time_days"], errors="coerce")

brca_survival_labels_clean = brca_survival[
    brca_survival["time_days"].notna() & (brca_survival["time_days"] > 0)
].copy()

print("Clinical survival shape (raw):", brca_survival.shape)
print("Clinical survival shape (clean):", brca_survival_labels_clean.shape)

This section merges cleaned survival outcomes with PAM50 labels, prints quality-control summaries, and saves the required CSV outputs.

In [ ]:
risk_pam50 = pam50_labels.merge(
    brca_survival_labels_clean[["case_id", "event", "time_days"]],
    on="case_id",
    how="inner",
)

print("risk_pam50 shape:", risk_pam50.shape)
print("Subtype counts:")
print(risk_pam50["pam50_subtype"].value_counts())
print("Event counts:")
print(risk_pam50["event"].value_counts().sort_index())
print("time_days min:", float(risk_pam50["time_days"].min()))
print("time_days max:", float(risk_pam50["time_days"].max()))

brca_survival_labels_clean.to_csv(survival_out_path, index=False)
risk_pam50.to_csv(risk_out_path, index=False)

print(f"Saved: {survival_out_path}")
print(f"Saved: {risk_out_path}")

This section plots Kaplan-Meier survival curves by PAM50 subtype (LumA, LumB, Basal, Her2), excludes Normal, runs overall and pairwise log-rank tests, and saves the KM figure.

In [ ]:
subtype_order = ["LumA", "LumB", "Basal", "Her2"]
km_data = risk_pam50[risk_pam50["pam50_subtype"].isin(subtype_order)].copy()
km_data["pam50_subtype"] = pd.Categorical(
    km_data["pam50_subtype"], categories=subtype_order, ordered=True
)

fig, ax = plt.subplots(figsize=(10, 6))
kmf = KaplanMeierFitter()

for subtype in subtype_order:
    mask = km_data["pam50_subtype"] == subtype
    if mask.sum() == 0:
        continue
    kmf.fit(
        durations=km_data.loc[mask, "time_days"],
        event_observed=km_data.loc[mask, "event"],
        label=subtype,
    )
    kmf.plot_survival_function(ax=ax, ci_show=False)

ax.set_title("TCGA-BRCA Kaplan-Meier Survival by PAM50 Subtype")
ax.set_xlabel("Time (days)")
ax.set_ylabel("Survival probability")
ax.grid(alpha=0.25)
fig.tight_layout()
fig.savefig(km_fig_path, dpi=300)
plt.show()

overall_logrank = multivariate_logrank_test(
    km_data["time_days"],
    km_data["pam50_subtype"],
    km_data["event"],
)
print(f"Overall log-rank p-value: {overall_logrank.p_value:.6g}")

pairwise_logrank = pairwise_logrank_test(
    km_data["time_days"],
    km_data["pam50_subtype"],
    km_data["event"],
)
print("\nPairwise log-rank results:")
print(pairwise_logrank.summary.to_string())

print(f"Saved: {km_fig_path}")

This section fits a penalized Cox proportional hazards model with LumA as the reference using one-hot PAM50 encoding, then prints the model summary.

In [ ]:
cox_df = km_data[["time_days", "event", "pam50_subtype"]].copy()

pam50_dummies = pd.get_dummies(cox_df["pam50_subtype"], prefix="pam50").astype(int)
if "pam50_LumA" in pam50_dummies.columns:
    pam50_dummies = pam50_dummies.drop(columns=["pam50_LumA"])

cox_input = pd.concat([cox_df[["time_days", "event"]], pam50_dummies], axis=1)

cph = CoxPHFitter(penalizer=0.1)
cph.fit(cox_input, duration_col="time_days", event_col="event")

print(cph.summary)

cox_summary = cph.summary.copy()
hr_gt1_subtypes = (
    cox_summary.loc[cox_summary["exp(coef)"] > 1]
    .index.str.replace("pam50_", "", regex=False)
    .tolist()
)
print("Subtypes with HR > 1 vs LumA:", hr_gt1_subtypes)

## Key Findings
- Cases used (LumA/LumB/Basal/Her2): **1053**
- Deaths in used set: **145**
- Overall log-rank p-value: **0.0464**
- Subtypes with hazard ratio > 1 vs LumA: **LumB, Basal, Her2**